# Predicting Uber Ride Fares

## Austin Nguyen & Hilary Le

## Dataset: Uber Fares Datasets from Kaggle &mdash; www.kaggle.com/datasets/yasserh/uber-fares-dataset 

In [1]:
import numpy as np

### Metrics Functions

In [2]:
# mean absolute error
def find_mae(y_true, y_pred): 
    return np.mean(np.abs(y_true - y_pred))

# mean squared error
def find_mse(y_true, y_pred): 
    return np.mean((y_true - y_pred)**2)

# root mean squared error
def find_rsme(y_true, y_pred):
    return np.sqrt(find_mse(y_true, y_pred))

### Linear Regression

### Ridge Regression

### Polynomial Regression

### Support Vector Regression

### Decision Tree Regression

### Random Forest Regression